# 0616098 黃秉茂 ML HW4 SVM and ANN

### import module

In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn

## Data Input

### read json

In [2]:
dataset = pd.read_json('train.json')

In [3]:
print(dataset)

          id      cuisine                                        ingredients
0      22675      italian  [1% low-fat cottage cheese, low-fat marinara s...
1      32288  southern_us  [brown sugar, salt, eggs, butter, chopped peca...
2      44406         thai  [red chili peppers, bell pepper, garlic, fish ...
3      29355     moroccan  [water, green tea leaves, tangerine, fresh min...
4      39350      chinese  [vegetable oil, chile sauce, tomato paste, gar...
...      ...          ...                                                ...
29769   2278     japanese  [soy sauce, sesame oil, garlic, sake, flour, g...
29770    474   vietnamese  [mint, garlic sauce, chinese chives, rice nood...
29771  44229       indian  [potatoes, vegetable broth, oil, cashew nuts, ...
29772  20311  southern_us  [butter, powdered sugar, cream cheese, soften,...
29773  32823      chinese  [savoy cabbage, dumpling wrappers, ginger, soy...

[29774 rows x 3 columns]


# Data Preprocessing

In [4]:
from sklearn.utils import shuffle
from sklearn import preprocessing

In [5]:
dataset = dataset.drop(columns=dataset.columns[0])

In [6]:
dataset_last_word = dataset.copy()
dataset_last_2_word = dataset.copy()

### limit the number of words in a string which is in the list in a column using DataFrame

In [7]:
word_limit = 1
dataset_last_word[dataset_last_word.columns[-1]] = dataset_last_word[dataset_last_word.columns[-1]].apply(lambda x : [' '.join(item.split()[-word_limit:]) for item in x])
word_limit = 2
dataset_last_2_word[dataset_last_2_word.columns[-1]] = dataset_last_2_word[dataset_last_2_word.columns[-1]].apply(lambda x : [' '.join(item.split()[-word_limit:]) for item in x])

In [8]:
print(dataset_last_word)

           cuisine                                        ingredients
0          italian  [cheese, sauce, starch, spinach, whites, nutme...
1      southern_us  [sugar, salt, eggs, butter, pecans, halves, ex...
2             thai  [peppers, pepper, garlic, sauce, sauce, ginger...
3         moroccan  [water, leaves, tangerine, mint, sugar, water,...
4          chinese  [oil, sauce, paste, cloves, cabbage, vinegar, ...
...            ...                                                ...
29769     japanese  [sauce, oil, garlic, sake, flour, pork, wrappe...
29770   vietnamese  [mint, sauce, chives, noodles, shrimp, lettuce...
29771       indian  [potatoes, broth, oil, nuts, pepper, paprika, ...
29772  southern_us                   [butter, sugar, soften, extract]
29773      chinese  [cabbage, wrappers, ginger, sauce, oil, shrimp...

[29774 rows x 2 columns]


## Shuffle the data

In [9]:
dataset_shuffle = shuffle(dataset)
dataset_last_word_shuffle = shuffle(dataset_last_word)
dataset_last_2_word_shuffle = shuffle(dataset_last_2_word)

In [10]:
x = dataset_shuffle[dataset_shuffle.columns[-1]]
x_last_word = dataset_last_word_shuffle[dataset_last_word_shuffle.columns[-1]]
x_last_2_word = dataset_last_2_word_shuffle[dataset_last_2_word_shuffle.columns[-1]]
y = dataset_shuffle[dataset_shuffle.columns[0]]
y_last_word = dataset_last_word_shuffle[dataset_last_word_shuffle.columns[0]]
y_last_2_word = dataset_last_2_word_shuffle[dataset_last_2_word_shuffle.columns[0]]

### one hot encode a series of lists in pandas

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

In [12]:
mlb = MultiLabelBinarizer()
X = pd.DataFrame(mlb.fit_transform(x), columns=mlb.classes_, index=x.index)
X_last_word = pd.DataFrame(mlb.fit_transform(x_last_word), columns=mlb.classes_, index=x_last_word.index)
X_last_2_word = pd.DataFrame(mlb.fit_transform(x_last_2_word), columns=mlb.classes_, index=x_last_2_word.index)

In [13]:
X = np.array(X)
X_last_word = np.array(X_last_word)
X_last_2_word = np.array(X_last_2_word)
Y = np.array(y)
Y_last_word = np.array(y_last_word)
Y_last_2_word = np.array(y_last_2_word)

In [14]:
print('no word limit', X.shape)
print('last word', X_last_word.shape)
print('last 2 word', X_last_2_word.shape)

(29774, 6231)
(29774, 1764)
(29774, 4909)


In [15]:
from sklearn.decomposition import PCA

In [16]:
def pca_with_proper_n_com(data):
    n_com = 9
    explained_ratio = 0
    explained_ratio_threshold = 80
    while explained_ratio < explained_ratio_threshold:
        n_com += 1
        pca = PCA(n_components=n_com)
        # calculate variance ratios
        pca.fit(data)
        cumulative_sum_of_variance_explained = np.cumsum(pca.explained_variance_ratio_ * 100)
        explained_ratio = cumulative_sum_of_variance_explained[-1]
    pca = PCA(n_components = n_com)
    data_pca = pca.fit_transform(data)
    return data_pca, n_com, explained_ratio

In [17]:
X_pca, X_proper_n_com, X_explained_ratio = pca_with_proper_n_com(X)
X_pca_last_word, X_proper_n_com_last_word, X_explained_ratio_last_word = pca_with_proper_n_com(X_last_word)
X_pca_last_2_word, X_proper_n_com_last_2_word, X_explained_ratio_last_2_word = pca_with_proper_n_com(X_last_2_word)
print('n_com:', X_proper_n_com, 'explained variance ratio:', X_explained_ratio)
print('n_com:', X_proper_n_com_last_word, 'explained variance ratio:', X_explained_ratio_last_word)
print('n_com:', X_proper_n_com_last_2_word, 'explained variance ratio:', X_explained_ratio_last_2_word)

n_com: 508 explained variance ratio: 80.02134297358639
n_com: 110 explained variance ratio: 80.03405704066961
n_com: 385 explained variance ratio: 80.00841017452635


In [18]:
print('no word limit', X_pca.shape)
print('last word', X_pca_last_word.shape)
print('last 2 word', X_pca_last_2_word.shape)

(29774, 508)
(29774, 110)
(29774, 385)
(29774, 6231)


## Model Construction

### import module

In [19]:
from sklearn.svm import LinearSVC

In [20]:
from sklearn.svm import SVC

### Support Vector Machine

In [21]:
linear_svc_model = LinearSVC()

In [22]:
svc_linear_model = SVC(kernel = 'linear')
svc_poly_model = SVC(kernel = 'poly')
svc_rbf_model = SVC(kernel = 'rbf')

# Validation

## Holdout validation 

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size=0.3, random_state=0)
x_train_last_word, x_val_last_word, y_train_last_word, y_val_last_word = train_test_split(X_last_word, Y_last_word, test_size=0.3, random_state=0)
x_train_last_2_word, x_val_last_2_word, y_train_last_2_word, y_val_last_2_word = train_test_split(X_last_2_word, Y_last_2_word, test_size=0.3, random_state=0)

x_train_pca, x_val_pca, y_train_pca, y_val_pca = train_test_split(X_pca, Y, test_size=0.3, random_state=0)
x_train_pca_last_word, x_val_pca_last_word, y_train_pca_last_word, y_val_pca_last_word = train_test_split(X_pca_last_word, Y_last_word, test_size=0.3, random_state=0)
x_train_pca_last_2_word, x_val_pca_last_2_word, y_train_pca_last_2_word, y_val_pca_last_2_word = train_test_split(X_pca_last_2_word, Y_last_2_word, test_size=0.3, random_state=0)

# Results

In [25]:
linear_svc = linear_svc_model.fit(x_train, y_train)
pred_linear_svc = linear_svc.predict(x_val)

linear_svc_last_word = linear_svc_model.fit(x_train_last_word, y_train_last_word)
pred_linear_svc_last_word = linear_svc_last_word.predict(x_val_last_word)

linear_svc_last_2_word = linear_svc_model.fit(x_train_last_2_word, y_train_last_2_word)
pred_linear_svc_last_2_word = linear_svc_last_2_word.predict(x_val_last_2_word)

In [26]:
svc_linear = svc_linear_model.fit(x_train_pca, y_train_pca)
pred_svc_linear = svc_linear.predict(x_val_pca)

svc_linear_last_word = svc_linear_model.fit(x_train_pca_last_word, y_train_pca_last_word)
pred_svc_linear_last_word = svc_linear_last_word.predict(x_val_pca_last_word)

svc_linear_last_2_word = svc_linear_model.fit(x_train_pca_last_2_word, y_train_pca_last_2_word)
pred_svc_linear_last_2_word = svc_linear_last_2_word.predict(x_val_pca_last_2_word)

In [27]:
svc_poly = svc_poly_model.fit(x_train_pca, y_train_pca)
pred_svc_poly = svc_poly.predict(x_val_pca)

svc_poly_last_word = svc_poly_model.fit(x_train_pca_last_word, y_train_pca_last_word)
pred_svc_poly_last_word = svc_poly_last_word.predict(x_val_pca_last_word)

svc_poly_last_2_word = svc_poly_model.fit(x_train_pca_last_2_word, y_train_pca_last_2_word)
pred_svc_poly_last_2_word = svc_poly.predict(x_val_pca_last_2_word)

In [28]:
svc_rbf = svc_rbf_model.fit(x_train_pca, y_train_pca)
pred_svc_rbf = svc_rbf.predict(x_val_pca)

svc_rbf_last_word = svc_rbf_model.fit(x_train_pca_last_word, y_train_pca_last_word)
pred_svc_rbf_last_word = svc_rbf.predict(x_val_pca_last_word)

svc_rbf_last_2_word = svc_rbf_model.fit(x_train_pca_last_2_word, y_train_pca_last_2_word)
pred_svc_rbf_last_2_word = svc_rbf_last_2_word.predict(x_val_pca_last_2_word)

In [29]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score
from sklearn.metrics import precision_recall_fscore_support

In [30]:
print('no word limit')
print('linear_svc', accuracy_score(y_val, pred_linear_svc))
print('svc_linear', accuracy_score(y_val_pca, pred_svc_linear))
print('svc_poly', accuracy_score(y_val_pca, pred_svc_poly))
print('svc_rbf', accuracy_score(y_val_pca, pred_svc_rbf))

print('last word')
print('linear_svc', accuracy_score(y_val_last_word, pred_linear_svc_last_word))
print('svc_linear', accuracy_score(y_val_pca_last_word, pred_svc_linear_last_word))
print('svc_poly', accuracy_score(y_val_pca_last_word, pred_svc_poly_last_word))
print('svc_rbf', accuracy_score(y_val_pca_last_word, pred_svc_rbf_last_word))

print('last 2 word')
print('linear_svc', accuracy_score(y_val_last_2_word, pred_linear_svc_last_2_word))
print('svc_linear', accuracy_score(y_val_pca_last_2_word, pred_svc_linear_last_2_word))
print('svc_poly', accuracy_score(y_val_pca_last_2_word, pred_svc_poly_last_2_word))
print('svc_rbf', accuracy_score(y_val_pca_last_2_word, pred_svc_rbf_last_2_word))

no word limit
linear_svc 0.7522668756296876
svc_linear 0.7056979738049928
svc_poly 0.5780812716892422
svc_rbf 0.7144296428971231
last word
linear_svc 0.7101757528265981
svc_linear 0.6495018470838464
svc_poly 0.637299899249972
svc_rbf 0.6763685212134781
last 2 word
linear_svc 0.7690585469607075
svc_linear 0.7204746445762902
svc_poly 0.616142393372887
svc_rbf 0.7276390910108587


# Comparison & Conclusion

- sklearn中 Decision Tree 使用的演算法是CART

- 普遍上Cross-Validation的表現比Holdout Validation好